In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import csv

In [24]:
dataDir='annotations'
dataType='train'
annFileImgNet='{}/ILSVRC2014/ILSVRC2014_{}.json'.format(dataDir,dataType)
annFileCOCO='{}/COCO2017/instances_{}2017.json'.format(dataDir,dataType)

In [25]:
coco = COCO(annFileCOCO)
#imgNet = COCO(annFileImgNet)

loading annotations into memory...
Done (t=11.11s)
creating index...
index created!


In [26]:
# display COCO categories 
catsCOCO = coco.loadCats(coco.getCatIds())
nmsCOCO=[cat['name'] for cat in catsCOCO]
#print('COCO categories: \n{}\n'.format(' '.join(nmsCOCO)))

#catsImgNet = imgNet.loadCats(imgNet.getCatIds())
#nmsImgNet=[cat['name'] for cat in catsImgNet]
#print(nmsImgNet)
#print('ILSVRC categories: \n{}\n'.format(' '.join(nmsImgNet)))

In [27]:
# wanted classes
allowed_classes = ['bottle', 'person', 'backpack', 'handbag', 'umbrella', 'bowl', 'cup', 'clock', 'knife', 'spoon', 'fork',
                   'chair', 'potted plant', 'dining table', 'keyboard', 'mouse', 'laptop', 'cell phone', 'scissors', 'book',
                  'monitor', 'coffee maker', 'filing cabinet', 'pencil box', 'pencil sharpener', 'lamp']

# mappings from old label to new label for consistency between datasets
mappings = {'monitor':'tv', 'mouse':'computer mouse','keyboard':'computer keyboard','cup':'cup or mug','cell phone':'iPod',
            'clock':'digital clock','potted plant':'flower pot','handbag':'purse','bottle':'water bottle'}



In [28]:
catIdsImgNet = []
catIdsCOCO = []
class_counts = {}
annotations = []
total_images = 0
#class_anns = {}

def add_cat_to_annotations(catIds, class_name, dataset):
    count = 0 # count of annotations in category
    img_count = 0 # count of images
    imgIds = dataset.getImgIds(catIds=catIds[-1])
    for imgId in imgIds:
        img_count += 1
        img = dataset.loadImgs(imgId)
        annIds = dataset.getAnnIds(imgIds=img[0]['id'], catIds=catIds[-1], iscrowd=None)
        anns = dataset.loadAnns(annIds)
        count += len(anns)
        for a in anns:
            # image filename, width, height, class, xmin, xmax, ymin, ymax
            info = {'file_name':img[0]['file_name'], 'width':img[0]['width'], 'height':img[0]['height'], 'class':c, 'xmin':a['bbox'][0], 'xmax':a['bbox'][0]+a['bbox'][2], 'ymin':a['bbox'][1], 'ymax':a['bbox'][1]+a['bbox'][3]}
            annotations.append(info)
            #if class_name not in class_anns:
            #    class_anns[class_name] = [info]
            #else:
            #    class_anns[class_name].append(info)
    return count, img_count

# ILSVRC dataset
# for each annId in each imgId in each object class, add to csv
'''for c in allowed_classes:
    if c in nmsImgNet:
        catIds = imgNet.getCatIds(catNms=c)
        count, img_count = add_cat_to_annotations(catIds, c, imgNet)
        total_images += img_count
        catIdsImgNet.append(catIds)
        class_counts[c] = count
    elif c in mappings and mappings[c] in nmsImgNet:
        catIds = imgNet.getCatIds(catNms=mappings[c])
        count, img_count = add_cat_to_annotations(catIds, c, imgNet)
        total_images += img_count
        catIdsImgNet.append(catIds)
        class_counts[c] = count
print(len(annotations))
print(class_counts)
print(len(catIdsImgNet))
print(total_images)'''

for c in allowed_classes:
    if c in nmsCOCO:
        catIds = coco.getCatIds(catNms=c)
        count, img_count = add_cat_to_annotations(catIds, c, coco)
        total_images += img_count
        catIdsCOCO.append(catIds)
        if c in class_counts:
            class_counts[c] += count
        else:
            class_counts[c] = count
    elif c in mappings and mappings[c] in nmsCOCO:
        catIds = coco.getCatIds(catNms=mappings[c])
        count, img_count = add_cat_to_annotations(catIds, c, coco)
        total_images += img_count
        catIdsCOCO.append(catIds)
        if c in class_counts:
            class_counts[c] += count
        else:
            class_counts[c] = count
print(len(annotations))
print(len(class_counts))
print(class_counts)
print(len(catIdsCOCO))
print(total_images)

491447
21
{'bottle': 24342, 'person': 262465, 'backpack': 8720, 'handbag': 12354, 'umbrella': 11431, 'bowl': 14358, 'cup': 20650, 'clock': 6334, 'knife': 7770, 'spoon': 6165, 'fork': 5479, 'chair': 38491, 'potted plant': 8652, 'dining table': 15714, 'keyboard': 2855, 'mouse': 2262, 'laptop': 4970, 'cell phone': 6434, 'scissors': 1481, 'book': 24715, 'monitor': 5805}
21
173543


In [29]:
# write to csv
with open('train_coco.csv', mode='w') as csv_file:
    fieldnames = ['file_name', 'width', 'height', 'class', 'xmin', 'xmax', 'ymin', 'ymax']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for ann in annotations:
        writer.writerow(ann)